In [ ]:
import pandas as pd
#import geopandas as gpd
import requests
import rasterio as rio
from rasterio.plot import show
from rasterio.warp import transform
from rasterio.transform import (xy, rowcol)
from rasterio.windows import Window
import itertools as it
import rioxarray as rx
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import time
import timeit
import os.path
from string import Template

from typing import Tuple, Optional, List

from leaf.deforestation import (
    cache_earthenginepartners_hansen,
    earthenginepartners_hansen)


In [ ]:
df = pd.read_csv('data/assets_with_deforestation.csv', sep='\t')

In [ ]:
df[df.treecover2000 > 0]

In [ ]:
df[df['2014'] != 0].head()

In [ ]:
# spanning the range 180W-180E and 80N-60S
# 80N, 70N, ... 10N, 00N, 10S, 20S, ... 50S - 180W
# 80N - 180W, 170W, ... 010W, 000E, 010E.... 170E

earthenginepartners_hansen('data/assets_for_deforestation.csv', '/t', range(-20, -31, -10), range(-50, -71, -10))

